# 1-Ontologías RDFS: Fundamentos

Estudiaremos la libreria `rdflib` basada en el lenguaje python, para la manipulación de metadatos en RDF.

Igualmente usaremos la libreria `owlrl` para implementar un razonador sobre ontologías RDFS que genere nuevo conocimiento mediante inferencias.

In [ ]:
# Instalaremos en nuestro entorno la siguiente libreria
!pip install rdflib owlrl

Ejemplo RDFS 1

Haciendo uso de RDFLib se crea el grafo y se construye las tripletas asociadas al vocabulario de RDFS que representa las clases y sus jerarquías.




In [ ]:
from rdflib import Graph, Namespace, URIRef, BNode, Literal
from rdflib.namespace import RDF, RDFS, FOAF, XSD
from rdflib.collection import Collection

# Crear grafo
g = Graph()

# Namespaces
EX = Namespace("http://example.org/animal#")
g.bind("ex", EX)
g.bind("foaf", FOAF)
g.bind("rdfs", RDFS)

# -----------------------------
# Ontología RDFS - Clases
# -----------------------------
g.add((EX.Animal, RDF.type, RDFS.Class))
g.add((EX.Mamifero, RDF.type, RDFS.Class))
g.add((EX.Ave, RDF.type, RDFS.Class))
g.add((EX.Habitat, RDF.type, RDFS.Class))  # 🏞️ Nueva clase: Habitat

# Jerarquía de clases
g.add((EX.Mamifero, RDFS.subClassOf, EX.Animal))
g.add((EX.Ave, RDFS.subClassOf, EX.Animal))


In [ ]:
# -----------------------------
# Propiedades con dominio y rango
# -----------------------------

# ex:tieneNombre
g.add((EX.tieneNombre, RDF.type, RDF.Property))
g.add((EX.tieneNombre, RDFS.subPropertyOf, FOAF.name))  # 👈 EX.tieneNombre subpropiedad de FOAF.name
g.add((EX.tieneNombre, RDFS.domain, EX.Animal))
g.add((EX.tieneNombre, RDFS.range, XSD.string))

# ex:edad
g.add((EX.edad, RDF.type, RDF.Property))
g.add((EX.edad, RDFS.domain, EX.Animal))
g.add((EX.edad, RDFS.range, XSD.integer))

# ex:especie
g.add((EX.especie, RDF.type, RDF.Property))
g.add((EX.especie, RDFS.domain, EX.Animal))
g.add((EX.especie, RDFS.range, XSD.string))

# ex:color
g.add((EX.color, RDF.type, RDF.Property))
g.add((EX.color, RDFS.domain, EX.Ave))
g.add((EX.color, RDFS.range, XSD.string))

# ex:numeroPatas
g.add((EX.numeroPatas, RDF.type, RDF.Property))
g.add((EX.numeroPatas, RDFS.domain, EX.Animal))
g.add((EX.numeroPatas, RDFS.range, XSD.integer))

# ex:alimentosFavoritos
g.add((EX.alimentosFavoritos, RDF.type, RDF.Property))
g.add((EX.alimentosFavoritos, RDFS.domain, EX.Animal))
g.add((EX.alimentosFavoritos, RDFS.range, RDF.List))

# 🌍 ex:habitaEn
g.add((EX.habitaEn, RDF.type, RDF.Property))
g.add((EX.habitaEn, RDFS.domain, EX.Animal))
g.add((EX.habitaEn, RDFS.range, EX.Habitat))  # 👈 range con valor de tipo clase Habitat


In [ ]:
# -----------------------------
# Instancias
# -----------------------------

# 🦁 León: Simba
leon = URIRef(EX.Leon)
g.add((leon, RDF.type, EX.Mamifero))
g.add((leon, EX.tieneNombre, Literal("Simba")))
g.add((leon, EX.edad, Literal(5, datatype=XSD.integer)))
g.add((leon, EX.especie, Literal("Panthera leo")))

# 🐘 Elefante: Dumbo
elefante = URIRef(EX.Elefante)
g.add((elefante, RDF.type, EX.Mamifero))
g.add((elefante, EX.tieneNombre, Literal("Dumbo")))
g.add((elefante, EX.edad, Literal(10, datatype=XSD.integer)))
g.add((elefante, EX.numeroPatas, Literal(4, datatype=XSD.integer)))

# Relación con una URI: Dumbo habita en la sabana
sabana = URIRef(EX.SabanaAfricana)
g.add((sabana, RDF.type, EX.Habitat))
g.add((elefante, EX.habitaEn, sabana))

# Lista RDF: Alimentos favoritos
comidas = BNode()
Collection(g, comidas, [
    Literal("Hojas"),
    Literal("Frutas"),
    Literal("Caña")
])
g.add((elefante, EX.alimentosFavoritos, comidas))

# 🐦 Pájaro anónimo
pajaro = BNode()
g.add((pajaro, RDF.type, EX.Ave))
g.add((pajaro, EX.tieneNombre, Literal("Pájaro sin nombre")))
g.add((pajaro, EX.color, Literal("Amarillo")))

# -----------------------------
# Guardar el grafo
# -----------------------------
g.serialize("ontologia_animales_rdfs.ttl", format="turtle")


Ejemplo RDFS 2

Se propone un ejemplo sencillo usando rdflib y owlrl para demostrar cómo DeductiveClosure(RDFS_Semantics) infiere nuevas tripletas. Se trabajará con una jerarquía de clases e instancias, y luego aplicar el razonador.

El ejemplo está en un contexto académico con estudiantes, personas y cursos. Se detallará cómo el razonador infiere relaciones usando la semántica de RDFS.

**Escenario:**

*   Estudiante es una subclase de Persona.
*   Tomar cursos tienen como dominio a Persona.

Se propone que el razonador infiera automáticamente que:

1.   Si Juan toma un curso, entonces (por dominio) también debe ser un Estudiante (podría inferirse si no se declarara explícitamente).
2.   Si Juan es un Estudiante por ser subclase de Persona, Juan también deberia ser una Persona (podría inferirse si no se declarara explícitamente).
3.   Si Juan toma un curso por ser esta subpropiedad de estar en Curso, entonces juan está en un curso tambien (podría inferirse si no se declarara explícitamente).


In [ ]:
pip install rdflib owlrl

In [ ]:
from rdflib import Graph, Namespace, RDF, RDFS, URIRef, Literal
from owlrl import DeductiveClosure, RDFS_Semantics

# Crear grafo RDF
g = Graph()
EX = Namespace("http://example.org/")
g.bind("ex", EX)

# Ontología
g.add((EX.Estudiante, RDFS.subClassOf, EX.Persona))  # Estudiante ⊆ Persona
g.add((EX.tomaCurso, RDFS.domain, EX.Estudiante))       # tomaCurso tiene dominio Estudiante
g.add((EX.Juan, EX.tomaCurso, EX.CursoAI))           # Juan toma el curso de IA
g.add((EX.tomaCurso, RDFS.subPropertyOf, EX.EstarCurso))           # JtomaCurso es subpropiedad de EstarCurso

print("=== Triples de tipos antes del razonamiento ===")
for s, p, o in g:
    if s == EX.Juan and p == RDF.type:
        print(f"{s} {p} {o}")


print("=== Triples de propiedades antes del razonamiento ===")
for s, p, o in g:
    if s == EX.Juan and o == EX.CursoAI:
        print(f"{s} {p} {o}")

DeductiveClosure(RDFS_Semantics, axiomatic_triples=True, datatype_axioms=False).expand(g)

# Ver resultados
print("=== Triples de tipos despues del razonamiento ===")
for s, p, o in g:
    if s == EX.Juan and p == RDF.type:
        print(f"{s} {p} {o}")

print("=== Triples de propiedades despues del razonamiento ===")
for s, p, o in g:
    if s == EX.Juan and o == EX.CursoAI:
        print(f"{s} {p} {o}")


In [ ]:
# -----------------------------
# Guardar el grafo
# -----------------------------
g.serialize("ontologia_academia_rdfs.ttl", format="turtle")
